# Canadian Tide Time Series Data Downloading

**1. Set up Enviroment**

In [1]:
!pip install requests
!pip install pandas
import requests
import pandas as pd
from datetime import datetime
from datetime import timedelta
import time

**2. User Input**

The script will ask for and validate the follow inputs:

- Station number
- Time Series Resolution
- Start Date
- End Date
- Tide Series

In [2]:
stn_num = int(input('Please Provide Station Number: '))
data_res = input('Provide number of the Resolution for Data Download 1, 3, 15, 60: ')
start_date = input('Provide Start Date in YYYY-MM-DD Format: ')
end_date = input('Provide End Date in YYYY-MM-DD Format: ')
tide_series = input('Provide Tide Series Code of wlo or wlp: ').lower()

# Check the user resolution input is valid
# Select format based on the resolution for the API endpoint
valid_res = False
while not valid_res:
    if data_res == '1':
        resolution = 'ONE_MINUTE'
        valid_res = True
    elif data_res == '3':
        resolution = 'THREE_MINUTES'
        valid_res = True
    elif data_res == '15':
        resolution = 'FIFTEEN_MINUTES'
        valid_res = True
    elif data_res == '60':
        resolution = 'SIXTY_MINUTES'
        valid_res = True
    else:
        data_res = input(f'Provided value was {data_res} and is not 1, 3, 15, or 60. Please provide a valid resolution')
        valid_res = False

# Check the user Start Date and End Date input is valid
valid_dates = False
while not valid_dates:
    try:
        format = '%Y-%m-%d'
        ma = datetime.strptime(end_date, format).date()
        mi = datetime.strptime(start_date, format).date()
    except ValueError:
        start_date = input(f'Provided value was {start_date}, please Provide a Start Date in YYYY-MM-DD Format: ')
        end_date = input(f'Provided value was {end_date}, please Provide a Start Date in YYYY-MM-DD Format: ')
        pass
    if  ma < mi:
        start_date = input(f'Start Date of {start_date} is not < End Date of {end_date}, please provide a Start Date that is less than the End Date ')
        end_date = input('Provide End Date in YYYY-MM-DD Format: ')
        valid_dates = False
    else:
        valid_dates = True
          
# Check the time series user input is valid
valid_timeseries = False
while not valid_timeseries:
    if (tide_series == 'wlo') or (tide_series == 'wlp'):
        valid_timeseries = True
    else:
        tide_series = input(f'Provided value was {tide_series}, please provide Tide Series Code of wlo or wlp: ').lower()
        valid_timeseries = False
    

**2. Find station endpoint based on station number provided**

In [3]:
# Correcting to have Station in 5 digit format
stn_num = f"{stn_num:05d}"
stn_url = f'https://api.iwls-sine.azure.cloud-nuage.dfo-mpo.gc.ca/api/v1/stations?code={stn_num}'
stn_url

'https://api.iwls-sine.azure.cloud-nuage.dfo-mpo.gc.ca/api/v1/stations?code=00300'

In [4]:
content = requests.get(stn_url)
if content.status_code == 200:
  df = pd.read_json(content.text)
  stn_id = df['id'].to_list()
else:
  print('Error with request')
stn_id

['5cebf1df3d0f4a073c4bbc62']

3. Create incremental list of Min Dates Max Date for API endpoints

In [5]:
dayoffset = 6
start_dates =[]
end_dates = []
while mi < ma:
    start_dates.append(mi.strftime(format))
    mi = mi + timedelta(days=dayoffset)
    end_dates.append(mi.strftime(format))
end_dates[-1] = end_date
print(start_dates)
print(end_dates)

['2024-10-10']
['2024-10-12']


3. Create a list urls to request Tidal data from the API grouped by 7 day intervalsrequest until end date limit reached

In [6]:
request_list = []

i = 0
for i in range(len(start_dates)):
  sd = start_dates[i]
  ed = end_dates[i]
  data_url = (f'https://api.iwls-sine.azure.cloud-nuage.dfo-mpo.gc.ca/api/v1/' +
                f'stations/5cebf1e33d0f4a073c4bc21f/data?time-series-code={tide_series}&from=' +
                f'{sd}T00%3A00%3A00Z&to={ed}T23%3A59%3A59Z&'+
                f'resolution={resolution}')
  request_list.append(data_url)


In [7]:
request_list

['https://api.iwls-sine.azure.cloud-nuage.dfo-mpo.gc.ca/api/v1/stations/5cebf1e33d0f4a073c4bc21f/data?time-series-code=wlp&from=2024-10-10T00%3A00%3A00Z&to=2024-10-12T23%3A59%3A59Z&resolution=THREE_MINUTES']

4. Append data to a new Dataframe and export as a csv

In [8]:
data = pd.DataFrame()
for url in request_list:
  content = requests.get(url)
  if content.status_code == 200:
    df = pd.read_json(content.text)
    data = pd.concat([data, df])
    time.sleep(1)
  else:
    print('Error with request')
data

data.to_csv(f'{stn_num}_{start_date}_{end_date}_{resolution}.csv')